In [6]:
import requests
import json
import pyspark
from delta import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder
    .appName("delta")
    .master("spark://spark-master:7077")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.defaultFS", "file:///")  # Define local como padrão
    .config("spark.hadoop.fs.s3a.access.key", "projeto_final")
    .config("spark.hadoop.fs.s3a.secret.key", "projeto_final")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
)
spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()





In [13]:

df_stops = (
    spark.read
    .option("multiLine", "true")
    .json("s3a://raw/sptrans/stops/")
)


In [12]:
df_prevision_stops.show()

+------------------------+-----+----+----+---+---+
|codigo_parada_solicitado|   hr|   p| ano|mes|dia|
+------------------------+-----+----+----+---+---+
|                   18848|19:20|null|2026|  1| 14|
|                   18849|19:20|null|2026|  1| 14|
|                   18850|19:20|null|2026|  1| 14|
|                   18851|19:20|null|2026|  1| 14|
|                   18852|19:20|null|2026|  1| 14|
|                   18853|19:20|null|2026|  1| 14|
|                   18854|19:20|null|2026|  1| 14|
|                   18855|19:20|null|2026|  1| 14|
|                   18856|19:20|null|2026|  1| 14|
|                   18857|19:20|null|2026|  1| 14|
|                   18858|19:20|null|2026|  1| 14|
|                   18859|19:20|null|2026|  1| 14|
|                   18860|19:20|null|2026|  1| 14|
|                   18861|19:20|null|2026|  1| 14|
|                   18862|19:20|null|2026|  1| 14|
|                   18863|19:20|null|2026|  1| 14|
|                   18864|19:20

In [14]:
df_stops.createOrReplaceTempView('stop')
df_stops_export = spark.sql(" SELECT A.*, current_date() dt_export from stop A")
df_stops_export

+---------+-------+----------+----------+----------------+----------+
|stop_desc|stop_id|  stop_lat|  stop_lon|       stop_name| dt_export|
+---------+-------+----------+----------+----------------+----------+
|     null|  18848|-23.554022|-46.671108|        Clínicas|2026-01-14|
|     null|  18849|-23.546498|-46.691141|   Vila Madalena|2026-01-14|
|     null|  18850|-23.558094|-46.660205|      Consolação|2026-01-14|
|     null|  18851|-23.635039|-46.641239|       Conceição|2026-01-14|
|     null|  18852|-23.646033|-46.641028|       Jabaquara|2026-01-14|
|     null|  18853|-23.625882|-46.640936|       São Judas|2026-01-14|
|     null|  18854|-23.618245|-46.639139|           Saúde|2026-01-14|
|     null|  18855|-23.610583|-46.637918| Praça Da Árvore|2026-01-14|
|     null|  18856|-23.598541|-46.636638|      Santa Cruz|2026-01-14|
|     null|  18857|-23.589359|-46.634677|    Vila Mariana|2026-01-14|
|     null|  18858|-23.567615|-46.649027|      Brigadeiro|2026-01-14|
|     null|  18859| 

In [15]:
from delta.tables import DeltaTable
delta_path = "s3a://trusted/sptrans/stops"
if DeltaTable.isDeltaTable(spark, delta_path):
    delta_table = DeltaTable.forPath(spark, delta_path)
else:
    df_stops_export.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("dt_export") \
        .save(delta_path)

    delta_table = DeltaTable.forPath(spark, delta_path)
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "s3a://trusted/sptrans/stops")


In [17]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "s3a://trusted/sptrans/stops")

(
    delta_table.alias("t")
    .merge(
        df_stops_export.alias("s"),
        """
        t.stop_lat = s.stop_lon
        AND t.stop_lat = s.stop_lon
        """
    )
    .whenMatchedUpdateAll()  # Atualiza a última posição
    .whenNotMatchedInsertAll()  # Insere se não existir
    .execute()
)
